# Spark Analysis

## 1. Initilaize spark session

In [1]:
print(spark)

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Bayes classification using Spark") \
    .getOrCreate()

## 2. Import sample flight data

In [6]:
inputs = 'gs://cloud-training-demos-ml/flights/tzcorr/all_flights-*'
flights = spark.read \
    .csv(inputs)

Py4JJavaError: An error occurred while calling o60.csv.
: java.io.IOException: Error accessing gs://cloud-training-demos-ml/flights/tzcorr
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getObject(GoogleCloudStorageImpl.java:1945)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getItemInfo(GoogleCloudStorageImpl.java:1851)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystem.listFileInfo(GoogleCloudStorageFileSystem.java:1065)
	at com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystemBase.listStatus(GoogleHadoopFileSystemBase.java:999)
	at org.apache.hadoop.fs.Globber.listStatus(Globber.java:77)
	at org.apache.hadoop.fs.Globber.doGlob(Globber.java:235)
	at org.apache.hadoop.fs.Globber.glob(Globber.java:149)
	at org.apache.hadoop.fs.FileSystem.globStatus(FileSystem.java:1976)
	at com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystemBase.globInte